# Training Pipeline :) 

In [ ]:
import os
import sys
sys.path = ['../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',] + sys.path
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tqdm import tqdm
import torch
import torchvision.models as models
import albumentations as A
import torch.nn as nn
from efficientnet_pytorch import model as enet
import random
from sklearn.model_selection import KFold, StratifiedKFold
!pip install -q timm
import timm

In [ ]:
def set_seed(seed = 0):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

seed = 42
random_state = set_seed(seed)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

In [ ]:
RTransform = A.Compose([
    A.Resize(128, 768, cv2.INTER_NEAREST)
])  

In [ ]:

class ClassificationDataset:
    
    def __init__(self, image_paths, targets, isTrain=True): 
        self.image_paths = image_paths
        self.targets = targets
        self.isTrain = isTrain

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):      
        image = np.load(self.image_paths[item])
        image1 = np.vstack(image).transpose((1, 0)).astype(np.float32)[:, :, np.newaxis]
        targets = self.targets[item]
        image1 = RTransform(image=image1)["image"]
        image1 = image1.transpose(2, 0, 1)
                
        return {
            "image1": torch.tensor(image1, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long),
        }        

In [ ]:
df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
print (df.shape)
df['img_path'] = df['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')

In [ ]:
df.head()

In [ ]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.enet.load_state_dict(torch.load(pretrained_model[backbone]))
        
        
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=3, bias=False)


    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.conv1(x)
        x = self.extract(x)
        x = self.myfc(x)
        
        return x

In [ ]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [ ]:
def train(data_loader, model, optimizer, device):
    model.train()
    
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        inputs1 = data["image1"]
        targets = data['targets']
        
        inputs1, targets_a, targets_b, lam = mixup_data(inputs1, targets.view(-1, 1), use_cuda=True)

        inputs1 = inputs1.to(device, dtype=torch.float)
        targets_a = targets_a.to(device, dtype=torch.float)
        targets_b = targets_b.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
        outputs = model(inputs1)
        loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
        loss.backward()
        optimizer.step()
        
def evaluate(data_loader, model, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            inputs1 = data["image1"]
            targets = data["targets"]
            
            inputs1 = inputs1.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.long)
            
            output = model(inputs1)
            output = torch.sigmoid(output)
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
            
    return final_outputs, final_targets

In [ ]:
paths = [
 'efficientnet-b0-08094119.pth',
 'efficientnet-b1-dbc7070a.pth',
 'efficientnet-b2-27687264.pth',
 'efficientnet-b3-c8376fa2.pth',
 'efficientnet-b4-e116e8b3.pth',
 'efficientnet-b5-586e6cc6.pth',
 'efficientnet-b6-c76e70fd.pth',
 'efficientnet-b7-dcc49843.pth',
]
pretrained_model = {
    'efficientnet-b0': '../input/efficientnet-pytorch/' + paths[0],
    'efficientnet-b1': '../input/efficientnet-pytorch/' + paths[1],
    'efficientnet-b2': '../input/efficientnet-pytorch/' + paths[2],
    'efficientnet-b3': '../input/efficientnet-pytorch/' + paths[3],
    'efficientnet-b4': '../input/efficientnet-pytorch/' + paths[4],
    'efficientnet-b5': '../input/efficientnet-pytorch/' + paths[5],
    'efficientnet-b6': '../input/efficientnet-pytorch/' + paths[6],
    'efficientnet-b7': '../input/efficientnet-pytorch/' + paths[7],
}


In [ ]:
class MetricNet(nn.Module):

    def __init__(self, backbone='efficientnet_b5', pretrained=True):
        super(MetricNet, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained, num_classes=1, in_chans = 1)
        
    def forward(self, x):
        x = self.backbone(x)
        return x

In [ ]:
#timm.list_models()

In [ ]:
device = "cuda"
baseline_name = 'efficientnet-b0'
model = enetv2(baseline_name, out_dim=1)
model.to(device)
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 7e-4)
n_epochs = 25
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)

In [ ]:
#df = df.sample(n = 500).reset_index(drop=True)

In [ ]:
X = df.img_path.values
Y = df.target.values
skf = StratifiedKFold(n_splits=5)
fold = 0

for train_index, test_index in skf.split(X, Y):
    train_images, valid_images = X[train_index], X[test_index]
    train_targets, valid_targets = Y[train_index], Y[test_index]
    break


In [ ]:
epochs = 12
Batch_Size = 16

train_dataset = ClassificationDataset(image_paths=train_images, targets=train_targets)
valid_dataset = ClassificationDataset(image_paths=valid_images, targets=valid_targets, isTrain=False)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=Batch_Size,shuffle=True, num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=Batch_Size,shuffle=False, num_workers=4)


In [ ]:
for epoch in range(epochs):
    
    if epoch % 5 == 0 and epoch > 1:
        torch.save(model.state_dict(),'aug_training_1_' + str(epoch) + '.pt')
        
    train(train_loader, model, optimizer, device=device)
    predictions, valid_targets = evaluate(valid_loader, model, device=device)
    roc_auc = metrics.roc_auc_score(valid_targets, predictions)
    print(f"Epoch={epoch}, Valid ROC AUC={roc_auc}")
    scheduler.step()
    

In [ ]:
torch.save(model.state_dict(),'fold1_training_1_25_ep.pt')

In [ ]:
submission = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
submission['img_path'] = submission['id'].apply(lambda x: f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy')

test_dataset = ClassificationDataset(image_paths=submission.img_path.values, targets=submission.target.values)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8)
test_predictions, test_targets = evaluate(test_loader, model, device=device)

In [ ]:
test_predictions = np.array(test_predictions)
submission.target = test_predictions[:, 0]
submission.drop(['img_path'], axis=1, inplace=True)

In [ ]:
submission.to_csv('fold1_submission.csv', index=False)
submission.head()